In [20]:
#importing neccessary libraries
import numpy as np
import pandas as pd
!pip install geopy
from geopy.geocoders import Nominatim #Convert and address into lat and long
#Matplotlib and associated plotting modules
import matplotlib.cm as cm 
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium #map rendering library

print('All set!')

All set!


Reading the dataframe we prepared in previous question from csv file into dataframe df

In [28]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_62a0a46b4ac147c1b8f3481581dba87c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='V5G6G3oCSkjsFiFNcJiN71-iP8Jn8CYI0tw-74U-Zrkv',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_62a0a46b4ac147c1b8f3481581dba87c.get_object(Bucket='capstoneproject-donotdelete-pr-cgcvvdd3emodc6',Key='TorontoGeoData.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Unnamed: 0,Postal Code,District,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [30]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [31]:
df.head()

,Postal Code,District,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Getting geographical coordinates of Toronto

In [32]:
toronto_add = 'Toronto, Ontario' #city address
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(toronto_add)
latitude=location.latitude
longitude=location.longitude
print("The coordinates of Toronto are {},{}".format(latitude,longitude))

The coordinates of Toronto are 43.6534817,-79.3839347


Creating a map Toronto city with all the postal codes

In [33]:
Toronto_map=folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postalcode,district,neighbourhood in zip(df['Latitude'],df['Longitude'],df['Postal Code'],df['District'],df['Neighbourhood']):
    label='Postal Code:{},Neighbourhoods:{},District:{}'.format(postalcode,neighbourhood,district)
    label=folium.Popup(label,parse_html=True)   
    folium.CircleMarker([lat,long],radius=5,color='blue',popup=label,fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=True).add_to(Toronto_map)
    

In [34]:
Toronto_map

Defining foursquare credentials

In [49]:
# @hidden_cell
CLIENT_ID = 'VXITT350NIAA4I1BUCNF1YDJTNIJFHGWU1O4QL35FQ51LJT1' # your Foursquare ID
CLIENT_SECRET = '55AFRIHTVOI2Q2V1VTMOTF0PICRVRPVVN1I4WR112EWQ5ETN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [36]:
import requests #library for handle requests
import json #library for handle json files
from pandas.io.json import json_normalize #transform json file into a pandas dataframe

We create a function to extract dataframe of top 100 venues within 500m radius of each of the postal codes in Toronto

In [42]:
limit=100
def getNearbyVenues(codes,names,latitudes,longitudes,radius=500):
    venues_list=[]
    for code,name,lat,lng in zip(codes,names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(code,name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],
                          v['venue']['location']['lng'],
                          v['venue']['categories'][0]['name'])for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Postal Code','Neighbourhood','Neighbourhood Latitude',
                          'Neighbourhood Longitude','Venue name','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

Toronto Venues dataframe

In [43]:
Toronto_Venues=getNearbyVenues(codes=df['Postal Code'],
                               names=df['Neighbourhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude'],radius=500)

In [45]:
Toronto_Venues.head()

,Postal Code,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue name,Venue Latitude,Venue Longitude,Venue Category
0,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,M1C,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,M1C,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [46]:
Toronto_Venues.shape

(2151, 8)

In [47]:
Toronto_Venues.groupby('Postal Code').count()[['Venue name']]

,Venue name
Postal Code,
M1B,1
M1C,3
M1E,8
M1G,3
M1H,8
M1J,1
M1K,5
M1L,8
M1M,2


Now we check the number of unique venue categories that were returned

In [48]:
print('There are {} unique venue categories'.format(len(Toronto_Venues['Venue Category'].unique())))

There are 269 unique venue categories


Now we analyze each neighbourhood by creating a dataframe with one hot encoding

In [51]:
Toronto_onehot=pd.get_dummies(Toronto_Venues['Venue Category'],prefix='',prefix_sep='')
#Add Neighbourhood and Postal Code colum back to Toronto_onehot dataframe
Toronto_onehot['Postal Code']=Toronto_Venues['Postal Code']
columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[columns]
Toronto_onehot.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
Toronto_onehot.shape

(2151, 270)

In [54]:
X=Toronto_Venues['Neighbourhood']

In [55]:
Toronto_onehot[-1]=X

In [56]:
Toronto_onehot=Toronto_onehot.rename({-1:'Neighborhoods'},axis=1)

In [57]:
Toronto_onehot.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhoods
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Malvern, Rouge"
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Rouge Hill, Port Union, Highland Creek"
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Rouge Hill, Port Union, Highland Creek"
3,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Rouge Hill, Port Union, Highland Creek"
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Guildwood, Morningside, West Hill"


In [58]:
col=[Toronto_onehot.columns[0]]+[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[1:-1])
Toronto_onehot=Toronto_onehot[col]

In [59]:
Toronto_onehot.head()

,Postal Code,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,"Malvern, Rouge",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1C,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's group rows by postal code by taking the mean of the frequency of occurrence of each category

In [63]:
Toronto_grouped=Toronto_onehot.groupby(['Postal Code','Neighborhoods'],sort=False).mean().reset_index()


In [64]:
Toronto_grouped.head()

,Postal Code,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,"Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, we print each neighborhood along with top 5 venues for first 10 postal codes

In [66]:
num=5
for code,nbd in zip(Toronto_grouped['Postal Code'][0:10],Toronto_grouped['Neighborhoods'][0:10]):
    print('----'+code+':'+nbd+'----')
    temp=Toronto_grouped[Toronto_grouped['Postal Code']==code].T.reset_index()
    temp=temp.iloc[2:]
    temp.columns=['venue category','freq']
    temp['freq']=temp['freq'].astype(float)
    temp.sort_values(by=['freq'],ascending=False,inplace=True)
    temp=temp.round({'freq':2})
    print(temp.reset_index(drop=True).head(num))

----M1B:Malvern, Rouge----
               venue category  freq
0        Fast Food Restaurant   1.0
1                 Men's Store   0.0
2  Modern European Restaurant   0.0
3           Mobile Phone Shop   0.0
4          Miscellaneous Shop   0.0
----M1C:Rouge Hill, Port Union, Highland Creek----
      venue category  freq
0                Bar  0.33
1        Golf Course  0.33
2       Home Service  0.33
3  Accessories Store  0.00
4      Metro Station  0.00
----M1E:Guildwood, Morningside, West Hill----
   venue category  freq
0    Intersection  0.12
1  Breakfast Spot  0.12
2            Bank  0.12
3      Restaurant  0.12
4  Medical Center  0.12
----M1G:Woburn----
               venue category  freq
0                 Coffee Shop  0.67
1           Korean Restaurant  0.33
2           Accessories Store  0.00
3               Metro Station  0.00
4  Modern European Restaurant  0.00
----M1H:Cedarbrae----
         venue category  freq
0      Hakka Restaurant  0.12
1    Athletics & Sports  0.12
2   Fri

We now define a function to sort values of the venue in descending order

In [67]:
def most_common_venues(row,num_of_values):
    row_sorted=row.iloc[2:].sort_values(ascending=False)
    return row_sorted.index.values[0:num_of_values]

We now create a dataframe to contain top 10 venues of each postal code
We first create the name of columns of such dataframe

In [68]:
num_of_values=10
indicators=['st','nd','rd']
columns=['PostalCode','Neighborhoods']
for ind in np.arange(num_of_values):
    try:
        columns.append('{}{} Most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))

Initializing a dataframe

In [70]:
Toronto_top=pd.DataFrame(columns=columns)

Filling in the dataframe

In [75]:
Toronto_top['Postal Code']=Toronto_grouped['Postal Code']
Toronto_top['Neighborhoods']=Toronto_grouped['Neighborhoods']

In [77]:
for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_top.iloc[ind][2:]=most_common_venues(Toronto_grouped.iloc[ind],num_of_values)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


ValueError: cannot set using a slice indexer with a different length than the value

In [74]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Postal Code
0,NaN,"Malvern, Rouge",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M1B
1,NaN,"Rouge Hill, Port Union, Highland Creek",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M1C
2,NaN,"Guildwood, Morningside, West Hill",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M1E
3,NaN,Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M1G
4,NaN,Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M1H
